[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/block_DCT_compression.ipynb)

# III... video compression

## Parameters

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
!ln -sf ~/MRVC/src/image_3.py .
import image_3 as image
!ln -sf ~/MRVC/src/image_1.py .
import image_1 as component
!ln -sf ~/MRVC/src/block_DCT.py .
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg as color
#!ln -sf ~/MRVC/src/color_DCT.py .
#import color_DCT as color
#!ln -sf ~/MRVC/src/RGB.py .
#import RGB as color
import cv2 # pip install opencv-python
!ln -sf ~/quantization/information.py .
import information
!ln -sf ~/quantization/distortion.py .
import distortion
import os
import pylab
!ln -sf ~/quantization/deadzone_quantizer.py .
import deadzone_quantizer as Q
import math
import block_DCT as DCT

In [ ]:
G = 2 # GOP size

In [ ]:
!~/MRVC/sequences/container/runme.sh -n $G

In [ ]:
sequence = "/tmp/original_"

In [ ]:
block_y_side = block_x_side = 8

In [ ]:
N_components = 3

In [ ]:
entropy_estimator = "PNG"
if entropy_estimator == "PNG":
    def compute_BPP(_image, filename_prefix, index):
        BPP = image.write(_image, filename_prefix, index)*8/_image.size
        return BPP
else:
    def compute_BPP(_image, filename_prefix, index):
        entropy = information.entropy(_image.flatten().astype(np.int16))
        return entropy

## Quantization steps

In [ ]:
Q_steps = [128, 64, 32, 16, 8]

## Using same $\Delta$ for all coefficients

### Version 0: Each (quantization indexes) decomposition is written in a different PNG file

In [ ]:
RD_points_no_RDO = []
for Q_step in Q_steps:
    acc_BPP = 0 # Accacumulated rate
    acc_MSE = 0 # Accumulated distortion
    for i in range(G):
        x = image.read(sequence, i)
        xx = color.from_RGB(x.astype(np.int16) - 128)
        yy = DCT.analyze_image(xx, block_y_side, block_x_side)
        yy_k = DCT.uniform_quantize(yy, block_y_side, block_x_side, N_components, Q_step)
        yy_dQ = DCT.uniform_dequantize(yy_k, block_y_side, block_x_side, N_components, Q_step)
        zz_dQ = DCT.synthesize_image(yy_dQ, block_y_side, block_x_side)
        z_dQ = color.to_RGB(zz_dQ) + 128
        MSE = distortion.MSE(x, z_dQ)
        acc_MSE += MSE
        yy_k_subbands = DCT.get_subbands(yy_k, block_y_side, block_x_side)
        assert (yy_k_subbands.all() >= -128), f"min value = {np.min(yy_k_subbands)}"
        assert (yy_k_subbands.all() <= 127), f"min value = {np.max(yy_k_subbands)}"
        if __debug__:
            print(np.min(yy_k_subbands), np.max(yy_k_subbands))
        BPP = compute_BPP((yy_k_subbands + 128).astype(np.uint8), f"/tmp/{Q_step}_", i)
        acc_BPP += BPP
    RD_points_no_RDO.append((acc_BPP/G, acc_MSE/G))
    print(i, Q_step, end=' ', flush=True)

In [ ]:
RD_points_no_RDO

### Version 1: All (quantization indexes) decomposition are concatenated and then written into a single PNG file

In [ ]:
RD_points_no_RDO_one_PNG = []
for Q_step in Q_steps:
    avg_MSE = 0
    sequence_of_quantized_decompositions = []
    for i in range(G):
        x = image.read(sequence, i)
        xx = color.from_RGB(x.astype(np.int16) - 128)
        yy = DCT.analyze_image(xx, block_y_side, block_x_side)
        yy_k = DCT.uniform_quantize(yy, block_y_side, block_x_side, N_components, Q_step)
        yy_dQ = DCT.uniform_dequantize(yy_k, block_y_side, block_x_side, N_components, Q_step)
        zz_dQ = DCT.synthesize_image(yy_dQ, block_y_side, block_x_side)
        z_dQ = color.to_RGB(zz_dQ) + 128
        MSE = distortion.MSE(x, z_dQ)
        avg_MSE += MSE
        yy_k_subbands = DCT.get_subbands(yy_k, block_y_side, block_x_side)
        sequence_of_quantized_decompositions.append(yy_k_subbands)
    concatenation = np.concatenate(sequence_of_quantized_decompositions)
    assert (concatenation.all() >= -128), f"min value = {np.min(concatenation)}"
    assert (concatenation.all() <= 127), f"min value = {np.max(concatenation)}"
    if __debug__:
        print(np.min(concatenation), np.max(concatenation))
    BPP = compute_BPP((concatenation + 128).astype(np.uint8), f"/tmp/{Q_step}_", 0)
    RD_points_no_RDO_one_PNG.append((BPP, avg_MSE/G))
    print(i, Q_step, end=' ', flush=True)

In [ ]:
RD_points_no_RDO_one_PNG

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_no_RDO), label="Different PNG files")
pylab.plot(*zip(*RD_points_no_RDO_one_PNG), label="One PNG file")
pylab.title("")
pylab.xlabel("BPP")
pylab.ylabel("MSE")
plt.legend(loc="best")
pylab.show()

The differences are insignificant.

## Using RDO (Rate/Distortion Optimization)

### Find the optimal progression of combinations of quantization steps
Each input frame is transformed. The resulting subband-components are quantized and their RD contribution estimated, supossing that the distortion can be measured in the transform domain, and the spatial/statistical decorrelation of the entropy codec between subband-components is zero.

In [ ]:
RD_points = []
RD_slopes = []
N_components = xx.shape[2]
single_list = []
counter = 0

for frame_number in range(G):
    x = image.read(sequence, frame_number)
    blocks_in_y = x.shape[0]//block_y_side
    blocks_in_x = x.shape[1]//block_x_side

    xx = color.from_RGB(x.astype(np.int16))
    xx[...,0] -= np.average(xx[...,0]).astype(np.int16)
    xx[...,1] -= np.average(xx[...,1]).astype(np.int16)
    xx[...,2] -= np.average(xx[...,2]).astype(np.int16)

    yy = DCT.analyze_image(xx, block_y_side, block_x_side)
    yy = DCT.get_subbands(yy, block_y_side, block_x_side)

    for _y in range(block_y_side):
        for _x in range(block_x_side):
            for _c in range(N_components):
                sbc = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                         blocks_in_x*_x : blocks_in_x*(_x + 1),
                         _c]
                sbc_energy = information.average_energy(sbc)
                # The first point of each RD curve has a maximum distortion equal
                # to the energy of the subband and a rate = 0
                RD_points.append([(0, sbc_energy)]) # (Rate, Distortion) of a subband-component of a frame
                RD_slopes.append([])
                counter += 1
    print(counter)
    #input()

    for _y in range(block_y_side):
        for _x in range(block_x_side):
            for _c in range(N_components):
                sbc = yy[blocks_in_y*_y : blocks_in_y*(_y + 1),
                         blocks_in_x*_x : blocks_in_x*(_x + 1),
                         _c]
                frame_subband_component_number = 0
                for Q_step in Q_steps:
                    sbc_k = Q.quantize(sbc, Q_step)
                    sbc_dQ = Q.dequantize(sbc_k, Q_step)
                    MSE = distortion.MSE(sbc, sbc_dQ)
                    assert (sbc_k.all() >= 0), f"min value = {np.min(sbc_k)}"
                    assert (sbc_k.all() <= 255), f"min value = {np.max(sbc_k)}"
                    BPP = component.write(sbc_k.astype(np.uint8), f"/tmp/{_y}_{_x}_{Q_step}_", 0)*8/xx.size
                    #BPP_Q_indexes = information.PNG_BPP((Q_indexes.astype(np.int32) + 32768).astype(np.uint16), "/tmp/BPP_")[0]
                    #BPP_Q_indexes = information.entropy(Q_indexes.astype(np.int16).flatten())
                    point = (BPP, MSE)
                    RD_points[frame_number*block_y_side*block_x_side*N_components + (_y*block_x_side + _x)*N_components + _c].append(point)
                    print("Q_step =", Q_step, "BPP =", point[0], "MSE =", point[1])
                    delta_BPP = BPP - RD_points[frame_number*block_y_side*block_x_side*N_components + (_y*block_x_side + _x)*N_components + _c][frame_subband_component_number][0]
                    delta_MSE = RD_points[frame_number*block_y_side*block_x_side*N_components + (_y*block_x_side + _x)*N_components + _c][frame_subband_component_number][1] - MSE
                    if delta_BPP > 0:
                        slope = delta_MSE/delta_BPP
                        RD_slopes[frame_number*block_y_side*block_x_side*N_components + (_y*block_x_side + _x)*N_components + _c].append((slope, (frame_number, _y, _x, _c), Q_step))
                    else:
                        slope = 0
                    frame_subband_component_number += 1

    def filter_slopes(slopes):
        filtered_slopes = []
        slopes_iterator = iter(slopes)
        prev = next(slopes_iterator)
        for curr in slopes_iterator:
            if prev[0] < curr[0]:
                print(f"deleted {prev}")
            else:
                filtered_slopes.append(prev)
            prev = curr
        filtered_slopes.append(prev)
        return filtered_slopes

    filtered_slopes = []
    for i in RD_slopes:
        filtered_slopes.append(filter_slopes(i))

    for l in filtered_slopes:
        #l = filter_slopes(l)
        for i in l:
            #if i[1] > 0:
            single_list.append(i)

In [ ]:
sorted_slopes = sorted(single_list, key=lambda x: x[0])[::-1]

In [ ]:
sorted_slopes

## Build the optimal RD curve
For each quantization steps combination, again, compute the distortion in the transform domain, but now the rate contribution is measured compressing all the decompositions.

In [ ]:
frame = image.read(sequence, frame_number) # Used only to get the shape
blocks_in_y = frame.shape[0]//block_y_side
blocks_in_x = frame.shape[1]//block_x_side
decomposed_GOF = []
for frame_number in range(G):
    frame = image.read(sequence, frame_number)
    YUV_frame = color.from_RGB(frame.astype(np.int16))
    YUV_frame[...,0] -= np.average(YUV_frame[...,0]).astype(np.int16)
    YUV_frame[...,1] -= np.average(YUV_frame[...,1]).astype(np.int16)
    YUV_frame[...,2] -= np.average(YUV_frame[...,2]).astype(np.int16)
    DCT_blocks = DCT.analyze_image(YUV_frame, block_y_side, block_x_side)
    DCT_decomposition = DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)
    decomposed_GOF.append(DCT_decomposition)

optimal_RD_points = []
Q_steps_combinations = []
reconstructed_decompositions = []
for i in range(G):
    reconstructed_decompositions.append(np.zeros_like(decomposed_GOF[0]))
    Q_steps_combinations.append(np.full(shape=(block_x_side, block_y_side, N_components), fill_value=99999999))
    
for s in sorted_slopes:
    sbc_index = s[1]
    decomposition = sbc_index[0]
    _y = sbc_index[1]
    _x = sbc_index[2]
    _c = sbc_index[3]
    reconstructed_decompositions
        [decomposition]
        [blocks_in_y*_y : blocks_in_y*(_y + 1), blocks_in_x*_x : blocks_in_x*(_x + 1), c]
        = decomposed_GOF[decomposition]
                        [blocks_in_y*_y : blocks_in_y*(_y + 1), blocks_in_x*_x : blocks_in_x*(_x + 1), _c]
    Q_steps_combination[decomposition][_y, _x, _c] = s[2]
    yy_prog_k = DCT.quantize(yy_prog, Q_steps_combination)
    yy_prog_dQ = DCT.dequantize(yy_prog_k, Q_steps_combination)
    
    MSE = distortion.MSE(yy, yy_prog_dQ)

    BPP = image.write((yy_prog_k + 128).astype(np.uint8), f"/tmp/{_y}_{_x}_{_c}_{s[0]}_", 0)*8/xx.size
    point = (BPP, MSE)
    print("sbc =", sbc_index, "Q_step =", s[2], "BPP =", BPP, "MSE =", MSE)
    optimal_RD_points.append(point)

## Compare

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_no_RDO), label="No RDO")
pylab.plot(*zip(*optimal_RD_points), label="Using RDO")
pylab.title("Effect of using RDO")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE")
plt.legend(loc="best")
#pylab.yscale('log')
#pylab.xscale('log')
pylab.show()